# Classification

In [13]:
import pandas as pd
import sys;sys.path.append('..');from thesis_package import aimodels as my_ai, utils, metrics
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import os
import optuna

## Max u

In [14]:
exogenous_data = pd.read_csv('..\data\processed\production\exogenous_data_extended.csv').drop(columns=['date'])
y_max_u_sparse_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_sparse_bool_constr.csv').drop(columns=['timestamps'])
y_max_u_sparse_bool = pd.DataFrame(y_max_u_sparse_bool['bus_16'], columns=['bus_16'])
# Testing data
train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data, y_max_u_sparse_bool, test_size=0.2, scaling=True)
data_max_class = {'X_train': train_x, 'X_test': valid_x, 'y_train': train_y, 'y_test': valid_y}

In [15]:
num_trials = 30

In [16]:
def objective(trial):
    # import data
    y_max_u_balanced_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_balanced_bool_constr.csv')
    y_max_u_balanced_bool = pd.DataFrame(y_max_u_balanced_bool['bus_16'], columns=['bus_16'])
    exogenous_data = pd.read_csv('..\data\ground_truth\\exogenous_data_vm_pu_max_balanced.csv').drop(columns=['date'])
    train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data, y_max_u_balanced_bool, test_size=0.2, scaling=True)
    data = {'X_train': train_x, 'X_test': valid_x, 'y_train': train_y, 'y_test': valid_y}
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
    model = my_ai.Context(my_ai.XGBoostClassifierStrategy(param))
    model.fit(data)
    prediction = model.predict(data_max_class)
    prediction = pd.DataFrame(prediction , columns=valid_y.columns)
    tp, tn, fp, fn = 0, 0, 0, 0
    tp += sum((prediction['bus_16'] == 1) & (data_max_class['y_test']['bus_16'] == 1))
    tn += sum((prediction['bus_16'] == 0) & (data_max_class['y_test']['bus_16'] == 0))
    fp += sum((prediction['bus_16'] == 1) & (data_max_class['y_test']['bus_16'] == 0))
    fn += sum((prediction['bus_16'] == 0) & (data_max_class['y_test']['bus_16'] == 1))
    if (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn) > 0:
        mcc_score = (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    else: 
        mcc_score = 0
    return mcc_score
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=num_trials)
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
# Write the results to a csv file.
with open("./hyper_params_results_mcc/params_xgboost_balanced_classifier_max_u_test_bus_16.csv", "w") as f:
    f.write("params,value\n")
    for key, value in trial.params.items():
        f.write("{},{}\n".format(key, value))
    f.write("value,{}\n".format(trial.value))

[I 2022-10-21 11:54:40,358] A new study created in memory with name: no-name-15dbeefd-eab2-4049-9473-0e512f919b2b
[I 2022-10-21 11:54:48,338] Trial 0 finished with value: 0.5714118338843156 and parameters: {'booster': 'dart', 'lambda': 1.8835371958220824e-07, 'alpha': 0.0011088719841518557, 'subsample': 0.38388708006168515, 'colsample_bytree': 0.4229607331161819, 'max_depth': 5, 'min_child_weight': 3, 'eta': 2.949719000498082e-08, 'gamma': 0.000951669624987712, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 5.29171444639736e-07, 'skip_drop': 2.52446204905082e-07}. Best is trial 0 with value: 0.5714118338843156.
[I 2022-10-21 11:54:53,150] Trial 1 finished with value: 0.5577008373282989 and parameters: {'booster': 'dart', 'lambda': 0.38092412350783944, 'alpha': 0.0109230063924983, 'subsample': 0.8200628263535812, 'colsample_bytree': 0.25117596916460444, 'max_depth': 9, 'min_child_weight': 8, 'eta': 0.003510497976179185, 'gamma': 5.93998556

Number of finished trials:  30
Best trial:
  Value: 0.6404548085149984
  Params: 
    booster: dart
    lambda: 0.14182485122786342
    alpha: 0.0001488031719695729
    subsample: 0.7911264853768102
    colsample_bytree: 0.6710795352112778
    max_depth: 9
    min_child_weight: 9
    eta: 0.11948410602614275
    gamma: 2.970868915721095e-08
    grow_policy: lossguide
    sample_type: weighted
    normalize_type: tree
    rate_drop: 0.06814776932706042
    skip_drop: 0.95055313893407


# Hybrid training and testing

In [17]:
# Classification data sparse
y_max_u_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_sparse_bool_constr.csv').drop(columns=['timestamps'])
y_max_u_bool = pd.DataFrame(y_max_u_bool['bus_16'], columns=['bus_16'])
train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data, y_max_u_bool, test_size=0.2, scaling=True)
data_max_u_bool = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}
# reg data sparse
y_max_u = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_constr.csv').drop(columns=['timestamps'])
y_max_u = pd.DataFrame(y_max_u['bus_16'], columns=['bus_16'])
exogenous_data = pd.read_csv('..\data\processed\production\exogenous_data_extended.csv').drop(columns=['date'])
train_x, valid_x, train_y, valid_y, scaler_max = utils.split_and_suffle(exogenous_data, y_max_u, test_size=0.2, scaling=True)
data_max_reg = {'X_train': deepcopy(train_x), 'X_test': deepcopy(valid_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(valid_y)}
threshold_max = utils.compute_threshold(y_max_u) / scaler_max['y']
# balanced data reg
y_max_u_balanced = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_balanced_constr.csv')
y_max_u_balanced = pd.DataFrame(y_max_u_balanced['bus_16'], columns=['bus_16'])
exogenous_data_vm_pu_max_balanced = pd.read_csv('..\data\ground_truth\\exogenous_data_vm_pu_max_balanced.csv').drop(columns=['date'])
X_max_u_train, X_max_u_test, y_max_u_train, y_max_u_test = utils.split_and_suffle(exogenous_data_vm_pu_max_balanced, y_max_u_balanced)
data_max_u_balanced = {'X_train': deepcopy(X_max_u_train), 'X_test': deepcopy(X_max_u_test), 'y_train': deepcopy(y_max_u_train), 'y_test': deepcopy(y_max_u_test)}
# class data bool balanced
y_max_u_balanced_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_balanced_bool_constr.csv')
y_max_u_balanced_bool = pd.DataFrame(y_max_u_balanced_bool['bus_16'], columns=['bus_16'])
exogenous_data_balanced = pd.read_csv('..\data\ground_truth\\exogenous_data_vm_pu_max_balanced.csv').drop(columns=['date'])
train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data_balanced, y_max_u_balanced_bool, test_size=0.2, scaling=True)
data_max_u_bool_balanced = {'X_train': train_x, 'X_test': valid_x, 'y_train': train_y, 'y_test': valid_y}
# hybrid data training
data_max_u_hybrid = {}
data_max_u_hybrid['X_train'] = pd.concat([data_max_u_balanced['X_train'], data_max_u_bool_balanced['y_train']], axis=1)
data_max_u_hybrid['y_train'] = deepcopy(data_max_u_balanced['y_train'])


In [18]:

# hybrid data test
import ast
def get_hyper_params_from_df(df):
    output = {}
    for row in df.iterrows():
        if row[1]['params'] != 'value':
            try:
                output[row[1]['params']] = ast.literal_eval(row[1]['value'])
            except :
                output[row[1]['params']] = row[1]['value']
    return output
# import classifier hyper params 
df = pd.read_csv('hyper_params_results_mcc/params_xgboost_balanced_classifier_max_u_test_bus_16.csv')
hyper_params = get_hyper_params_from_df(df)
classifier_max_u = my_ai.Context(strategy=my_ai.XGBoostClassifierStrategy(hyper_params)) 
classifier_max_u.fit(data_max_u_bool_balanced)
class_prediction = classifier_max_u.predict(data_max_u_bool).to_csv('class_prediction_max_u.csv', index=False)
data_max_u_hybrid['X_test']  = pd.concat([data_max_reg['X_test'], class_prediction], axis=1)
data_max_u_hybrid['y_test'] = deepcopy(data_max_reg['y_test'])


# # 
# if 'hybrid_regressor_max_u.pickle' not in os.listdir('pickles\hybrid_models_benchmark'):
#     print('training max u hybrid model... wait for it...')
#     hybrid_regressor_max_u = my_ai.Context(strategy=my_ai.GradientBoostRegressorStrategy(hyper_params)) 
#     hybrid_regressor_max_u.fit(data=data_max_u_hybrid)
#     utils.serialize_object('pickles\hybrid_models_benchmark\hybrid_regressor_max_u_one_bus', hybrid_regressor_max_u)
# else: 
#     print('loading max u hybrid model...')
#     hybrid_regressor_max_u = utils.deserialize_object('pickles\hybrid_models_benchmark\hybrid_regressor_max_u_one_bus')


In [19]:
print('data_max_u_hybrid X_train shape: ', data_max_u_hybrid['X_train'].shape)
print('data_max_u_hybrid y_train shape: ', data_max_u_hybrid['y_train'].shape)
print('data_max_u_hybrid X_test shape: ', data_max_u_hybrid['X_test'].shape)
print('data_max_u_hybrid y_test shape: ', data_max_u_hybrid['y_test'].shape)


data_max_u_hybrid X_train shape:  (5561, 12)
data_max_u_hybrid y_train shape:  (5561, 1)
data_max_u_hybrid X_test shape:  (9044, 11)
data_max_u_hybrid y_test shape:  (9044, 1)


In [20]:
num_trials = 30
# Same implementation as above, but for Gradient Boosting Regression.
def objective(trial):
    # import data
    y_max_u = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_balanced_constr.csv')
    y_max_u = pd.DataFrame(y_max_u['bus_16'], columns=['bus_16'])
    exogenous_data = pd.read_csv('..\data\ground_truth\\exogenous_data_vm_pu_max_balanced.csv').drop(columns=['date'])
    train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data, y_max_u, test_size=0.2, scaling=True)
    data = {'X_train': deepcopy(train_x), 'X_test': deepcopy(valid_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(valid_y)}
    # class data bool balanced
    y_max_u_balanced_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_balanced_bool_constr.csv')
    y_max_u_balanced_bool = pd.DataFrame(y_max_u_balanced_bool['bus_16'], columns=['bus_16'])
    train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data, y_max_u_balanced_bool, test_size=0.2, scaling=True)
    data_max_u_bool_balanced = {'X_train': deepcopy(train_x), 'X_test': deepcopy(valid_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(valid_y)}
    # hybrid data training
    data_max_u_hybrid = {}
    data_max_u_hybrid['X_train'] =  pd.concat([data['X_train'], data_max_u_bool_balanced['y_train']], axis=1)
    data_max_u_hybrid['y_train'] = data['y_train']
    # Testing data
    class_prediction = pd.read_csv('class_prediction_max_u.csv')
    data_max_u_hybrid['X_test']  = pd.concat([data_max_reg['X_test'], class_prediction], axis=1)
    data_max_u_hybrid['y_test'] = data_max_reg['y_test']
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 1.0, log=True) ,
        'loss': trial.suggest_categorical('loss', ['squared_error', 'absolute_error'])
    }
    model = my_ai.Context(my_ai.GradientBoostRegressorStrategy(param))
    model.fit(data_max_u_hybrid)
    prediction = model.predict(data_max_u_hybrid)
    # evaluate the regression performance with my metrics
    metric = metrics.Metrics()
    metric.get_prediction_scores(prediction, data_max_u_hybrid['y_test'], threshold=threshold_max)
    print('hybrid_recall', metric.hybrid_recall)
    print('hybrid_precision', metric.hybrid_precision)
    print('hybrid_f1', metric.hybrid_f1)
    print('hybrid_accuracy', metric.hybrid_accuracy)
    print('hybrid_mcc', metric.hybrid_mcc)
    return metric.hybrid_mcc
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=num_trials)
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
# Write the results to a csv file.
with open("./hyper_params_results_mcc/params_gradient_boost_regression_balanced_max_u_test_one_bus.csv", "w") as f:
    f.write("params,value\n")
    for key, value in trial.params.items():
        f.write("{},{}\n".format(key, value))
    f.write("value,{}\n".format(trial.value))

[I 2022-10-21 11:55:52,501] A new study created in memory with name: no-name-e199fcd9-c8be-4118-9b13-fdecc5c04c25
[I 2022-10-21 11:56:02,280] Trial 0 finished with value: 0.4002266609671506 and parameters: {'n_estimators': 417, 'learning_rate': 0.35977006145588475, 'loss': 'squared_error'}. Best is trial 0 with value: 0.4002266609671506.


true_positives_ctr:  591
true_negatives_ctr:  7315
false_positives_ctr:  1035
false_negatives_ctr:  103
69896159893200
hybrid_recall 0.7765614215615608
hybrid_precision 0.2719271446503238
hybrid_f1 0.4028048312892285
hybrid_accuracy 0.8491732157762555
hybrid_mcc 0.4002266609671506


[I 2022-10-21 11:56:15,829] Trial 1 finished with value: 0.25309767453785365 and parameters: {'n_estimators': 623, 'learning_rate': 0.5356408547957667, 'loss': 'squared_error'}. Best is trial 0 with value: 0.4002266609671506.


true_positives_ctr:  545
true_negatives_ctr:  6633
false_positives_ctr:  1717
false_negatives_ctr:  149
88898888691600
hybrid_recall 0.6857393435463653
hybrid_precision 0.1719975824556419
hybrid_f1 0.2750155804401447
hybrid_accuracy 0.756767255065997
hybrid_mcc 0.25309767453785365


[I 2022-10-21 11:56:20,495] Trial 2 finished with value: 0.5281841316632668 and parameters: {'n_estimators': 160, 'learning_rate': 0.12869537094053457, 'loss': 'absolute_error'}. Best is trial 2 with value: 0.5281841316632668.


...

true_positives_ctr:  590
true_negatives_ctr:  7797
false_positives_ctr:  553
false_negatives_ctr:  104
52332832100700
hybrid_recall 0.7748102159497631
hybrid_precision 0.40323962263245605
hybrid_f1 0.5304260802197778
hybrid_accuracy 0.9122972523512657
hybrid_mcc 0.5190606697925915


[I 2022-10-21 11:58:39,169] Trial 28 finished with value: 0.5141810552555246 and parameters: {'n_estimators': 278, 'learning_rate': 0.12981730358890015, 'loss': 'absolute_error'}. Best is trial 25 with value: 0.5295599848691138.


true_positives_ctr:  591
true_negatives_ctr:  7775
false_positives_ctr:  575
false_negatives_ctr:  103
53230491085200
hybrid_recall 0.7777460321124988
hybrid_precision 0.39569038255952527
hybrid_f1 0.5245220297117894
hybrid_accuracy 0.9099643550963987
hybrid_mcc 0.5141810552555246


[I 2022-10-21 11:58:50,955] Trial 29 finished with value: 0.5106876856106236 and parameters: {'n_estimators': 414, 'learning_rate': 0.3710401933193387, 'loss': 'absolute_error'}. Best is trial 25 with value: 0.5295599848691138.


true_positives_ctr:  577
true_negatives_ctr:  7816
false_positives_ctr:  534
false_negatives_ctr:  117
51073716228700
hybrid_recall 0.7489019482843028
hybrid_precision 0.4060736177817824
hybrid_f1 0.5266073715125358
hybrid_accuracy 0.9126528259937773
hybrid_mcc 0.5106876856106236
Number of finished trials:  30
Best trial:
  Value: 0.5295599848691138
  Params: 
    n_estimators: 164
    learning_rate: 0.128508806750761
    loss: absolute_error


In [21]:
# Same implementation as above, but for Gradient Boosting Regression.
def objective(trial):
    # import data
    y_max_u = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_max_balanced_constr.csv')
    y_max_u = pd.DataFrame(y_max_u['bus_16'], columns=['bus_16'])
    exogenous_data = pd.read_csv('..\data\ground_truth\\exogenous_data_vm_pu_max_balanced.csv').drop(columns=['date'])
    train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data, y_max_u, test_size=0.2, scaling=True)
    data = {'X_train': train_x, 'X_test': valid_x, 'y_train': train_y, 'y_test': valid_y}
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 1.0, log=True) ,
        'loss': trial.suggest_categorical('loss', ['squared_error', 'absolute_error'])
    }
    model = my_ai.Context(my_ai.GradientBoostRegressorStrategy(param))
    model.fit(data)
    prediction = model.predict(data_max_reg)
    # evaluate the regression performance with my metrics
    metric = metrics.Metrics()
    metric.get_prediction_scores(prediction, data_max_reg['y_test'], threshold=threshold_max)
    print('hybrid_recall', metric.hybrid_recall)
    print('hybrid_precision', metric.hybrid_precision)
    print('hybrid_f1', metric.hybrid_f1)
    print('hybrid_accuracy', metric.hybrid_accuracy)
    print('hybrid_mcc', metric.hybrid_mcc)
    return metric.hybrid_mcc
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=num_trials)
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
# Write the results to a csv file.
with open("./hyper_params_results_mcc/params_gradient_boost_regression_balanced_max_u_one_bus_not hybrid.csv", "w") as f:
    f.write("params,value\n")
    for key, value in trial.params.items():
        f.write("{},{}\n".format(key, value))
    f.write("value,{}\n".format(trial.value))

[I 2022-10-21 11:58:51,057] A new study created in memory with name: no-name-545cfb24-d30c-4655-b82d-be79559c7b63
[I 2022-10-21 11:59:04,171] Trial 0 finished with value: 0.5200193024610237 and parameters: {'n_estimators': 399, 'learning_rate': 0.14344219081822132, 'loss': 'absolute_error'}. Best is trial 0 with value: 0.5200193024610237.


true_positives_ctr:  588
true_negatives_ctr:  7803
false_positives_ctr:  547
false_negatives_ctr:  106
52019165753500
hybrid_recall 0.7717488363399436
hybrid_precision 0.40601020119024916
hybrid_f1 0.5320916933361907
hybrid_accuracy 0.9130892239876028
hybrid_mcc 0.5200193024610237


[I 2022-10-21 11:59:11,520] Trial 1 finished with value: 0.5264442861839969 and parameters: {'n_estimators': 171, 'learning_rate': 0.11717159309268428, 'loss': 'absolute_error'}. Best is trial 1 with value: 0.5264442861839969.


true_positives_ctr:  623
true_negatives_ctr:  7696
false_positives_ctr:  654
false_negatives_ctr:  71
57476478059100
hybrid_recall 0.8439509769288046
hybrid_precision 0.379817817064974
hybrid_f1 0.5238695729784729
hybrid_accuracy 0.9041353316883544
hybrid_mcc 0.5264442861839969


[I 2022-10-21 11:59:18,300] Trial 2 finished with value: 0.3789623052475237 and parameters: {'n_estimators': 103, 'learning_rate': 0.6079779367052507, 'loss': 'squared_error'}. Best is trial 1 with value: 0.5264442861839969.


...

true_positives_ctr:  675
true_negatives_ctr:  7422
false_positives_ctr:  928
false_negatives_ctr:  19
69121120992700
hybrid_recall 0.9575093698269693
hybrid_precision 0.32217763102302993
hybrid_f1 0.4821305526246683
hybrid_accuracy 0.8755078751772197
hybrid_mcc 0.513889208776058


[I 2022-10-21 12:02:17,058] Trial 28 finished with value: 0.4932355070774898 and parameters: {'n_estimators': 26, 'learning_rate': 0.14989203091611797, 'loss': 'absolute_error'}. Best is trial 21 with value: 0.5274255263066426.


true_positives_ctr:  689
true_negatives_ctr:  7253
false_positives_ctr:  1097
false_negatives_ctr:  5
75118060181200
hybrid_recall 0.9895636220980973
hybrid_precision 0.2914137669893984
hybrid_f1 0.45023817789118376
hybrid_accuracy 0.8559737558284418
hybrid_mcc 0.4932355070774898


[I 2022-10-21 12:02:19,773] Trial 29 finished with value: 0.5106271384203627 and parameters: {'n_estimators': 54, 'learning_rate': 0.11965308228402907, 'loss': 'absolute_error'}. Best is trial 21 with value: 0.5274255263066426.


true_positives_ctr:  675
true_negatives_ctr:  7411
false_positives_ctr:  939
false_negatives_ctr:  19
69492556698000
hybrid_recall 0.9573041317315628
hybrid_precision 0.3187878153415933
hybrid_f1 0.47829922204609054
hybrid_accuracy 0.8740908806935468
hybrid_mcc 0.5106271384203627
Number of finished trials:  30
Best trial:
  Value: 0.5274255263066426
  Params: 
    n_estimators: 247
    learning_rate: 0.1285425904126249
    loss: absolute_error


## Min u

In [22]:
exogenous_data = pd.read_csv('..\data\processed\production\exogenous_data_extended.csv').drop(columns=['date'])
y_min_u_sparse_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_sparse_bool_constr.csv').drop(columns=['timestamps'])
y_min_u_sparse_bool = pd.DataFrame(y_min_u_sparse_bool['bus_16'], columns=['bus_16'])
# Testing data
train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data, y_min_u_sparse_bool, test_size=0.2, scaling=True)
data_min_class = {'X_train': train_x, 'X_test': valid_x, 'y_train': train_y, 'y_test': valid_y}

In [23]:
def objective(trial):
    # import data
    y_min_u_balanced_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_balanced_bool_constr.csv')
    y_min_u_balanced_bool = pd.DataFrame(y_min_u_balanced_bool['bus_16'], columns=['bus_16'])
    exogenous_data = pd.read_csv('..\data\ground_truth\\exogenous_data_vm_pu_min_balanced.csv').drop(columns=['date'])
    train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data, y_min_u_balanced_bool, test_size=0.2, scaling=True)
    data = {'X_train': train_x, 'X_test': valid_x, 'y_train': train_y, 'y_test': valid_y}
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
    model = my_ai.Context(my_ai.XGBoostClassifierStrategy(param))
    model.fit(data)
    prediction = model.predict(data_min_class)
    prediction = pd.DataFrame(prediction , columns=valid_y.columns)
    tp, tn, fp, fn = 0, 0, 0, 0
    tp += sum((prediction['bus_16'] == 1) & (data_min_class['y_test']['bus_16'] == 1))
    tn += sum((prediction['bus_16'] == 0) & (data_min_class['y_test']['bus_16'] == 0))
    fp += sum((prediction['bus_16'] == 1) & (data_min_class['y_test']['bus_16'] == 0))
    fn += sum((prediction['bus_16'] == 0) & (data_min_class['y_test']['bus_16'] == 1))
    if (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn) > 0:
        mcc_score = (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    else: 
        mcc_score = 0
    return mcc_score
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=num_trials)
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
# Write the results to a csv file.
with open("./hyper_params_results_mcc/params_xgboost_balanced_classifier_min_u_test_bus_16.csv", "w") as f:
    f.write("params,value\n")
    for key, value in trial.params.items():
        f.write("{},{}\n".format(key, value))
    f.write("value,{}\n".format(trial.value))

[I 2022-10-21 12:02:21,868] A new study created in memory with name: no-name-a7aba0fd-ad2b-4b23-ab87-76443f8eae36
[I 2022-10-21 12:02:25,639] Trial 0 finished with value: 0.4174425022397923 and parameters: {'booster': 'gbtree', 'lambda': 0.0004956780420458229, 'alpha': 0.09512710689043151, 'subsample': 0.7749314446370554, 'colsample_bytree': 0.9697391306473191, 'max_depth': 5, 'min_child_weight': 3, 'eta': 3.486310198546269e-05, 'gamma': 0.005375091177485281, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.4174425022397923.
[I 2022-10-21 12:02:31,670] Trial 1 finished with value: 0.46786591953863593 and parameters: {'booster': 'dart', 'lambda': 1.3453075168902139e-05, 'alpha': 6.291305213032538e-07, 'subsample': 0.523907089727154, 'colsample_bytree': 0.8381216848105304, 'max_depth': 7, 'min_child_weight': 4, 'eta': 7.798974546525333e-05, 'gamma': 2.6470631526961524e-06, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.00015488

Number of finished trials:  30
Best trial:
  Value: 0.5611822753355297
  Params: 
    booster: dart
    lambda: 0.022918347917256048
    alpha: 0.00739342179575011
    subsample: 0.9822013746871794
    colsample_bytree: 0.5445470368014637
    max_depth: 7
    min_child_weight: 7
    eta: 0.03787709447887554
    gamma: 1.981572606347494e-05
    grow_policy: lossguide
    sample_type: uniform
    normalize_type: tree
    rate_drop: 8.473403749879414e-06
    skip_drop: 1.0678716691937125e-08


In [24]:
# Classification data sparse
y_min_u_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_sparse_bool_constr.csv').drop(columns=['timestamps'])
y_min_u_bool = pd.DataFrame(y_min_u_bool['bus_16'], columns=['bus_16'])
train_x, test_x, train_y, test_y, scaler = utils.split_and_suffle(exogenous_data, y_min_u_bool, test_size=0.2, scaling=True)
data_min_u_bool = {'X_train': deepcopy(train_x), 'X_test': deepcopy(test_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(test_y), 'scaler': deepcopy(scaler)}
# reg data sparse
y_min_u = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_constr.csv').drop(columns=['timestamps'])
y_min_u = pd.DataFrame(y_min_u['bus_16'], columns=['bus_16'])
exogenous_data = pd.read_csv('..\data\processed\production\exogenous_data_extended.csv').drop(columns=['date'])
train_x, valid_x, train_y, valid_y, scaler_min = utils.split_and_suffle(exogenous_data, y_min_u, test_size=0.2, scaling=True)
data_min_reg = {'X_train': deepcopy(train_x), 'X_test': deepcopy(valid_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(valid_y)}
threshold_min = utils.compute_threshold(y_min_u) / scaler_min['y']
# balanced data reg
y_min_u_balanced = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_balanced_constr.csv')
y_min_u_balanced = pd.DataFrame(y_min_u_balanced['bus_16'], columns=['bus_16'])
exogenous_data_vm_pu_min_balanced = pd.read_csv('..\data\ground_truth\\exogenous_data_vm_pu_min_balanced.csv').drop(columns=['date'])
X_min_u_train, X_min_u_test, y_min_u_train, y_min_u_test = utils.split_and_suffle(exogenous_data_vm_pu_min_balanced, y_min_u_balanced)
data_min_u_balanced = {'X_train': deepcopy(X_min_u_train), 'X_test': deepcopy(X_min_u_test), 'y_train': deepcopy(y_min_u_train), 'y_test': deepcopy(y_min_u_test)}
# class data bool balanced
y_min_u_balanced_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_balanced_bool_constr.csv')
y_min_u_balanced_bool = pd.DataFrame(y_min_u_balanced_bool['bus_16'], columns=['bus_16'])
exogenous_data_balanced = pd.read_csv('..\data\ground_truth\\exogenous_data_vm_pu_min_balanced.csv').drop(columns=['date'])
train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data_balanced, y_min_u_balanced_bool, test_size=0.2, scaling=True)
data_min_u_bool_balanced = {'X_train': train_x, 'X_test': valid_x, 'y_train': train_y, 'y_test': valid_y}
# hybrid data training
data_min_u_hybrid = {}
data_min_u_hybrid['X_train'] = pd.concat([data_min_u_balanced['X_train'], data_min_u_bool_balanced['y_train']], axis=1)
data_min_u_hybrid['y_train'] = deepcopy(data_min_u_balanced['y_train'])


In [25]:

# hybrid data test
import ast
def get_hyper_params_from_df(df):
    output = {}
    for row in df.iterrows():
        if row[1]['params'] != 'value':
            try:
                output[row[1]['params']] = ast.literal_eval(row[1]['value'])
            except :
                output[row[1]['params']] = row[1]['value']
    return output
# import classifier hyper params 
df = pd.read_csv('hyper_params_results_mcc/params_xgboost_balanced_classifier_min_u_test_bus_16.csv')
hyper_params = get_hyper_params_from_df(df)
classifier_min_u = my_ai.Context(strategy=my_ai.XGBoostClassifierStrategy(hyper_params)) 
classifier_min_u.fit(data_min_u_bool_balanced)
class_prediction = classifier_min_u.predict(data_min_u_bool).to_csv('class_prediction_min_u.csv', index=False)
data_min_u_hybrid['X_test']  = pd.concat([data_min_reg['X_test'], class_prediction], axis=1)
data_min_u_hybrid['y_test'] = deepcopy(data_min_reg['y_test'])

In [26]:
print('data_min_u_hybrid X_train shape: ', data_min_u_hybrid['X_train'].shape)
print('data_min_u_hybrid y_train shape: ', data_min_u_hybrid['y_train'].shape)
print('data_min_u_hybrid X_test shape: ', data_min_u_hybrid['X_test'].shape)
print('data_min_u_hybrid y_test shape: ', data_min_u_hybrid['y_test'].shape)


data_min_u_hybrid X_train shape:  (11164, 12)
data_min_u_hybrid y_train shape:  (11164, 1)
data_min_u_hybrid X_test shape:  (9044, 11)
data_min_u_hybrid y_test shape:  (9044, 1)


In [27]:
num_trials = 30
# Same implementation as above, but for Gradient Boosting Regression.
def objective(trial):
    # import data
    y_min_u = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_balanced_constr.csv')
    y_min_u = pd.DataFrame(y_min_u['bus_16'], columns=['bus_16'])
    exogenous_data = pd.read_csv('..\data\ground_truth\\exogenous_data_vm_pu_min_balanced.csv').drop(columns=['date'])
    train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data, y_min_u, test_size=0.2, scaling=True)
    data = {'X_train': deepcopy(train_x), 'X_test': deepcopy(valid_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(valid_y)}
    # class data bool balanced
    y_min_u_balanced_bool = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_balanced_bool_constr.csv')
    y_min_u_balanced_bool = pd.DataFrame(y_min_u_balanced_bool['bus_16'], columns=['bus_16'])
    train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data, y_min_u_balanced_bool, test_size=0.2, scaling=True)
    data_min_u_bool_balanced = {'X_train': deepcopy(train_x), 'X_test': deepcopy(valid_x), 'y_train': deepcopy(train_y), 'y_test': deepcopy(valid_y)}
    # hybrid data training
    data_min_u_hybrid = {}
    data_min_u_hybrid['X_train'] =  pd.concat([data['X_train'], data_min_u_bool_balanced['y_train']], axis=1)
    data_min_u_hybrid['y_train'] = data['y_train']
    # Testing data
    class_prediction = pd.read_csv('class_prediction_min_u.csv')
    data_min_u_hybrid['X_test']  = pd.concat([data_min_reg['X_test'], class_prediction], axis=1)
    data_min_u_hybrid['y_test'] = data_min_reg['y_test']
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 1.0, log=True) ,
        'loss': trial.suggest_categorical('loss', ['squared_error', 'absolute_error'])
    }
    model = my_ai.Context(my_ai.GradientBoostRegressorStrategy(param))
    model.fit(data_min_u_hybrid)
    prediction = model.predict(data_min_u_hybrid)
    # evaluate the regression performance with my metrics
    metric = metrics.Metrics()
    metric.get_prediction_scores(prediction, data_min_u_hybrid['y_test'], threshold=threshold_min)
    print('hybrid_recall', metric.hybrid_recall)
    print('hybrid_precision', metric.hybrid_precision)
    print('hybrid_f1', metric.hybrid_f1)
    print('hybrid_accuracy', metric.hybrid_accuracy)
    print('hybrid_mcc', metric.hybrid_mcc)
    return metric.hybrid_mcc
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=num_trials)
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
# Write the results to a csv file.
with open("./hyper_params_results_mcc/params_gradient_boost_regression_balanced_min_u_test_one_bus.csv", "w") as f:
    f.write("params,value\n")
    for key, value in trial.params.items():
        f.write("{},{}\n".format(key, value))
    f.write("value,{}\n".format(trial.value))

[I 2022-10-21 12:05:28,752] A new study created in memory with name: no-name-78f35e5d-6a18-480b-98b2-d6af85cc41ec
[I 2022-10-21 12:05:59,645] Trial 0 finished with value: 0.35550036638169635 and parameters: {'n_estimators': 801, 'learning_rate': 0.755274254012659, 'loss': 'squared_error'}. Best is trial 0 with value: 0.35550036638169635.


true_positives_ctr:  851
true_negatives_ctr:  6332
false_positives_ctr:  1690
false_negatives_ctr:  171
135472758029532
hybrid_recall 0.7799995430208047
hybrid_precision 0.2658186217859917
hybrid_f1 0.396509470760239
hybrid_accuracy 0.7612030721530926
hybrid_mcc 0.35550036638169635


[I 2022-10-21 12:06:01,652] Trial 1 finished with value: 0.3760505210156553 and parameters: {'n_estimators': 18, 'learning_rate': 0.12024991973499621, 'loss': 'squared_error'}. Best is trial 1 with value: 0.3760505210156553.


true_positives_ctr:  1008
true_negatives_ctr:  5504
false_positives_ctr:  2518
false_negatives_ctr:  14
159513541594512
hybrid_recall 0.9808235210681171
hybrid_precision 0.2283099141158045
hybrid_f1 0.37040036664565557
hybrid_accuracy 0.6793032610394036
hybrid_mcc 0.3760505210156553


[I 2022-10-21 12:06:04,341] Trial 2 finished with value: 0.5305681974796951 and parameters: {'n_estimators': 43, 'learning_rate': 0.18663688526902306, 'loss': 'absolute_error'}. Best is trial 2 with value: 0.5305681974796951.


...

true_positives_ctr:  993
true_negatives_ctr:  6428
false_positives_ctr:  1594
false_negatives_ctr:  29
136949600143356
hybrid_recall 0.9614851032500823
hybrid_precision 0.3132221669657114
hybrid_f1 0.47251389331804594
hybrid_accuracy 0.7890683844174888
hybrid_mcc 0.47459450908453094


[I 2022-10-21 12:07:36,762] Trial 28 finished with value: 0.49005287075360365 and parameters: {'n_estimators': 56, 'learning_rate': 0.4386730353471732, 'loss': 'absolute_error'}. Best is trial 23 with value: 0.5487099680412768.


true_positives_ctr:  967
true_negatives_ctr:  6654
false_positives_ctr:  1368
false_negatives_ctr:  55
128433474079260
hybrid_recall 0.9274646670475292
hybrid_precision 0.33935281160774594
hybrid_f1 0.49689516877129714
hybrid_accuracy 0.8138791544108972
hybrid_mcc 0.49005287075360365


[I 2022-10-21 12:08:03,767] Trial 29 finished with value: 0.34042104630329206 and parameters: {'n_estimators': 696, 'learning_rate': 0.7051596801306376, 'loss': 'squared_error'}. Best is trial 23 with value: 0.5487099680412768.


true_positives_ctr:  895
true_negatives_ctr:  5974
false_positives_ctr:  2048
false_negatives_ctr:  127
147205772451612
hybrid_recall 0.8324527123571264
hybrid_precision 0.2391432527106037
hybrid_f1 0.3715494567922626
hybrid_accuracy 0.7215450446331251
hybrid_mcc 0.34042104630329206
Number of finished trials:  30
Best trial:
  Value: 0.5487099680412768
  Params: 
    n_estimators: 14
    learning_rate: 0.3872266856076122
    loss: absolute_error


In [28]:
# Same implementation as above, but for Gradient Boosting Regression.
def objective(trial):
    # import data
    y_min_u = pd.read_csv('..\data\ground_truth\\res_bus_vm_pu_min_balanced_constr.csv')
    y_min_u = pd.DataFrame(y_min_u['bus_16'], columns=['bus_16'])
    exogenous_data = pd.read_csv('..\data\ground_truth\\exogenous_data_vm_pu_min_balanced.csv').drop(columns=['date'])
    train_x, valid_x, train_y, valid_y, scaler = utils.split_and_suffle(exogenous_data, y_min_u, test_size=0.2, scaling=True)
    data = {'X_train': train_x, 'X_test': valid_x, 'y_train': train_y, 'y_test': valid_y}
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 1.0, log=True) ,
        'loss': trial.suggest_categorical('loss', ['squared_error', 'absolute_error'])
    }
    model = my_ai.Context(my_ai.GradientBoostRegressorStrategy(param))
    model.fit(data)
    prediction = model.predict(data_min_reg)
    # evaluate the regression performance with my metrics
    metric = metrics.Metrics()
    metric.get_prediction_scores(prediction, data_min_reg['y_test'], threshold=threshold_min)
    print('hybrid_recall', metric.hybrid_recall)
    print('hybrid_precision', metric.hybrid_precision)
    print('hybrid_f1', metric.hybrid_f1)
    print('hybrid_accuracy', metric.hybrid_accuracy)
    print('hybrid_mcc', metric.hybrid_mcc)
    return metric.hybrid_mcc
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=num_trials)
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
# Write the results to a csv file.
with open("./hyper_params_results_mcc/params_gradient_boost_regression_balanced_min_u_one_bus_not hybrid.csv", "w") as f:
    f.write("params,value\n")
    for key, value in trial.params.items():
        f.write("{},{}\n".format(key, value))
    f.write("value,{}\n".format(trial.value))

[I 2022-10-21 12:08:03,838] A new study created in memory with name: no-name-519b5a7c-0c3f-4eba-beae-73b3cb7de3cd
[I 2022-10-21 12:08:05,900] Trial 0 finished with value: 0.5613640127535421 and parameters: {'n_estimators': 25, 'learning_rate': 0.9747796838178895, 'loss': 'absolute_error'}. Best is trial 0 with value: 0.5613640127535421.


true_positives_ctr:  974
true_negatives_ctr:  7000
false_positives_ctr:  1022
false_negatives_ctr:  48
115334698803072
hybrid_recall 0.9375450351637
hybrid_precision 0.4085789025755639
hybrid_f1 0.5691320254295182
hybrid_accuracy 0.8581472219387513
hybrid_mcc 0.5613640127535421


[I 2022-10-21 12:08:20,162] Trial 1 finished with value: 0.45418007206006566 and parameters: {'n_estimators': 329, 'learning_rate': 0.6656574171461921, 'loss': 'absolute_error'}. Best is trial 0 with value: 0.5613640127535421.


true_positives_ctr:  973
true_negatives_ctr:  6399
false_positives_ctr:  1623
false_negatives_ctr:  49
137234489263872
hybrid_recall 0.93495269923778
hybrid_precision 0.3031306384477307
hybrid_f1 0.4578250914323635
hybrid_accuracy 0.783148038626672
hybrid_mcc 0.45418007206006566


[I 2022-10-21 12:08:42,860] Trial 2 finished with value: 0.4315288125411382 and parameters: {'n_estimators': 543, 'learning_rate': 0.11095980486905928, 'loss': 'absolute_error'}. Best is trial 0 with value: 0.5613640127535421.


...

true_positives_ctr:  987
true_negatives_ctr:  6851
false_positives_ctr:  1171
false_negatives_ctr:  35
121829373858192
hybrid_recall 0.9544419668313655
hybrid_precision 0.3834354636666347
hybrid_f1 0.5470858387358475
hybrid_accuracy 0.8423142122495885
hybrid_mcc 0.5440545775404398


[I 2022-10-21 12:11:10,658] Trial 28 finished with value: 0.5411236912083278 and parameters: {'n_estimators': 23, 'learning_rate': 0.20101575981605033, 'loss': 'absolute_error'}. Best is trial 0 with value: 0.5613640127535421.


true_positives_ctr:  989
true_negatives_ctr:  6829
false_positives_ctr:  1193
false_negatives_ctr:  33
122754949907856
hybrid_recall 0.9566823332294038
hybrid_precision 0.37907657876970036
hybrid_f1 0.5429959891598464
hybrid_accuracy 0.8403981221119703
hybrid_mcc 0.5411236912083278


[I 2022-10-21 12:11:13,168] Trial 29 finished with value: 0.4653264592960774 and parameters: {'n_estimators': 40, 'learning_rate': 0.8044486591442415, 'loss': 'absolute_error'}. Best is trial 0 with value: 0.5613640127535421.


true_positives_ctr:  966
true_negatives_ctr:  6518
false_positives_ctr:  1504
false_negatives_ctr:  56
133125179525520
hybrid_recall 0.9255157400595314
hybrid_precision 0.3163437366930763
hybrid_f1 0.47152051107150406
hybrid_accuracy 0.796056339619123
hybrid_mcc 0.4653264592960774
Number of finished trials:  30
Best trial:
  Value: 0.5613640127535421
  Params: 
    n_estimators: 25
    learning_rate: 0.9747796838178895
    loss: absolute_error
